In [1]:
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator

import numpy as np 
import pandas as pd
import os
import seaborn as sns
import matplotlib.pyplot as plt
plt.style.use('ggplot')
from tqdm import tqdm
from progressbar import ProgressBar
import re
from scipy.cluster.vq import kmeans, vq
from pylab import plot, show
from matplotlib.lines import Line2D
import matplotlib.colors as mcolors
from sklearn.cluster import KMeans
from sklearn import neighbors
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import isbnlib
import goodreads_api_client as gr
from newspaper import Article

In [2]:
df = pd.read_csv('books.csv', error_bad_lines = False)

/var/folders/_r/wftd7dk93nj_hbzpqfw4vnsr0000gn/T/ipykernel_74009/397317604.py:1: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  df = pd.read_csv('books.csv', error_bad_lines = False)
b'Skipping line 3350: expected 12 fields, saw 13\nSkipping line 4704: expected 12 fields, saw 13\nSkipping line 5879: expected 12 fields, saw 13\nSkipping line 8981: expected 12 fields, saw 13\n'


In [4]:
df.head(2)

,bookID,title,authors,average_rating,isbn,isbn13,language_code,num_pages,ratings_count,text_reviews_count,publication_date,publisher
0,1,Harry Potter and the Half-Blood Prince (Harry ...,J.K. Rowling/Mary GrandPré,4.57,0439785960,9780439785969,eng,652,2095690,27591,9/16/2006,Scholastic Inc.
1,2,Harry Potter and the Order of the Phoenix (Har...,J.K. Rowling/Mary GrandPré,4.49,0439358078,9780439358071,eng,870,2153167,29221,9/1/2004,Scholastic Inc.


In [5]:
def final_df(df1, l):
    year_df = pd.DataFrame(l, columns=['Year'])
    df1 = df1.reset_index(drop=True)
    final = df1[['authors', 'average_rating', 'title']].join(year_df)
    return final

In [38]:
def bookdata(df):
    year=[]
    pbar = ProgressBar()
    for isbn in pbar(df.isbn13):
        try:
            details = isbnlib.meta(isbn)
            year.append(details['Year'])
        except :
            #Trying out with goodreads api now
            print("exception #1")
            try: 
                book_detail = client.book.show_by_isbn(isbn)
                keys_wanted = ['publication_year']
                reduced_book = {k:v for k,v in book_detail.items() if k in keys_wanted}
                year.append((reduced_book['publication_year']))
            
            except: 
                #Going with webscraping
                print("exception #2")
                try:
                    y = html(isbn)
                    year_extracted = reg(y) #Extracting year with regex
                    year.append(y)
                except:
                    print("exception #3")
                    year.append('0')
                
    return year

In [7]:
def plot_author_chart(author_df):
    year = bookdata(author_df)
    author_df = final_df(author_df, year)
    author_df.dropna(0, inplace=True)
    author_df = author_df[author_df['Year'].str.isnumeric()]
    author_df = author_df.set_index('title')
    author_df = author_df[author_df.Year !='0']
    plt.figure(figsize=(15,15))
    sns.set_context('talk')
    plt.xticks(rotation=30)
    ax =  sns.barplot(data=author_df, x='Year', y='average_rating')
    ax.set_title("Average rating of books over time, "+ author_df.authors[1])
    plt.xticks(rotation=30)
    return ax

In [8]:
authors= ['Stephen King', 'Agatha Christie', 'Dan Brown', 'J.K. Rowling']

In [9]:
author_df = df[df['authors']==authors[1]]
author_df = author_df[author_df['language_code']=='eng']
author_df.head(1)

,bookID,title,authors,average_rating,isbn,isbn13,language_code,num_pages,ratings_count,text_reviews_count,publication_date,publisher
4516,16297,Cards on the Table (Hercule Poirot #15),Agatha Christie,3.92,0425205959,9780425205952,eng,324,25236,1046,7/5/2005,Berkley


In [17]:
df.isnull().sum()

bookID                0
title                 0
authors               0
average_rating        0
isbn                  0
isbn13                0
language_code         0
  num_pages           0
ratings_count         0
text_reviews_count    0
publication_date      0
publisher             0
dtype: int64

In [13]:
details

{'ISBN-13': '9780425205952',
 'Title': 'Cards On The Table',
 'Authors': ['Agatha Christie'],
 'Publisher': 'Berkley Publishing Group',
 'Year': '2005',
 'Language': 'en'}

In [18]:
year=[]

In [12]:
details = isbnlib.meta('9780425205952')

In [15]:
year.append(details['Year'])

In [16]:
year

['2005']

In [29]:
details = isbnlib.meta('9780425205952')

In [30]:
details

{'ISBN-13': '9780425205952',
 'Title': 'Cards On The Table',
 'Authors': ['Agatha Christie'],
 'Publisher': 'Berkley Publishing Group',
 'Year': '2005',
 'Language': 'en'}

In [34]:
pbar = ProgressBar()
working_list = ["9780439358071"]
#for isbn in pbar(df.isbn13):
for isbn in working_list:
    try:
        print(isbn)
        details = isbnlib.meta(isbn)
        print(details['Year'])
        #details = isbnlib.meta(isbn)
        #year.append(details['Year'])
    except :
        print("Didn't Work")

9780439358071
Didn't Work


In [33]:
df.head(3)

,bookID,title,authors,average_rating,isbn,isbn13,language_code,num_pages,ratings_count,text_reviews_count,publication_date,publisher
0,1,Harry Potter and the Half-Blood Prince (Harry ...,J.K. Rowling/Mary GrandPré,4.57,0439785960,9780439785969,eng,652,2095690,27591,9/16/2006,Scholastic Inc.
1,2,Harry Potter and the Order of the Phoenix (Har...,J.K. Rowling/Mary GrandPré,4.49,0439358078,9780439358071,eng,870,2153167,29221,9/1/2004,Scholastic Inc.
2,4,Harry Potter and the Chamber of Secrets (Harry...,J.K. Rowling,4.42,0439554896,9780439554893,eng,352,6333,244,11/1/2003,Scholastic


In [35]:
not_working_list = ['9780439358071']

In [46]:
book_detail = gr.client.resources.Book.show_by_isbn(isbn='9780439358071')
book_detail

TypeError: show_by_isbn() missing 1 required positional argument: 'self'

In [48]:
def html(isbn):
    url = 'https://isbndb.com/book/'+isbn
    article = Article(url)
    #article = 'https://isbndb.com/book/9780450524684'
    article.download()
    article.parse()
    ar = article.html
    ar = ar[9300:9900]
    return ar

In [51]:
def reg(l):
    return re.search(r'(\b\d{4})\b',l).groups()[0]

In [52]:
y = html(not_working_list[0])
year_extracted = reg(y) #Extracting year with regex
year.append(y)
year

AttributeError: 'NoneType' object has no attribute 'groups'

In [ ]:
a